In [1]:
import numpy as np

from fast_slic import Slic
from PIL import Image

img = "/content/data/semi_sup_data/CWFID/num30/test/input/001_image.png"
with Image.open(img) as f:
   image = np.array(f)
# import cv2; image = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)   # You can convert the image to CIELAB space if you need.
slic = Slic(num_components=1600, compactness=10)
assignment = slic.iterate(image) # Cluster Map
print(assignment)
print(slic.slic_model.clusters) # The cluster information of superpixels.

[[   0    0    0 ...   46   46   46]
 [   0    0    0 ...   46   46   46]
 [   0    0    0 ...   46   46   46]
 ...
 [1515 1515 1515 ... 1527 1527 1527]
 [1515 1515 1515 ... 1527 1527 1527]
 [1515 1515 1515 ... 1527 1527 1527]]
[{'number': 0, 'yx': (11.0, 17.0), 'color': (52.0, 118.0, 134.0), 'num_members': 265}, {'number': 1, 'yx': (14.0, 55.0), 'color': (51.0, 135.0, 123.0), 'num_members': 359}, {'number': 2, 'yx': (11.0, 88.0), 'color': (61.0, 129.0, 126.0), 'num_members': 273}, {'number': 3, 'yx': (20.0, 116.0), 'color': (62.0, 135.0, 122.0), 'num_members': 415}, {'number': 4, 'yx': (13.0, 147.0), 'color': (45.0, 112.0, 139.0), 'num_members': 303}, {'number': 5, 'yx': (16.0, 180.0), 'color': (70.0, 120.0, 133.0), 'num_members': 384}, {'number': 6, 'yx': (21.0, 205.0), 'color': (64.0, 133.0, 124.0), 'num_members': 180}, {'number': 7, 'yx': (17.0, 239.0), 'color': (68.0, 123.0, 131.0), 'num_members': 361}, {'number': 8, 'yx': (14.0, 276.0), 'color': (63.0, 113.0, 138.0), 'num_members

In [3]:

import cv2
from skimage.segmentation import mark_boundaries 
from PIL import Image 
import matplotlib.pyplot as plt 
# fig = plt.figure() 
# plt.ion()
# ax = fig.add_subplot(1, 1, 1) 
# ax.set_title("Superpixels -- %d segments" % (160)) 
# ax.imshow(mark_boundaries(image, assignment)) 
# plt.axis("off") 
# plt.show()
slic_img = mark_boundaries(image, assignment)
x_pos, y_pos, w, h = cv2.selectROI('location', slic_img, False)
print(x_pos, y_pos)
print(w, h)



: 

: 

In [4]:
assignment.shape

(966, 1296)

In [4]:
# https://github.dev/yhenon/pyimgsaliency/blob/master/pyimgsaliency/saliency.py
# https://github.com/lihezhang-dut/AMC_AE
import cv2
import math
import sys
import operator
import networkx as nx
#import matplotlib.pyplot as plt
import numpy as np
import scipy.spatial.distance
import scipy.signal
import skimage
import skimage.io
from skimage.segmentation import slic
from fast_slic import Slic
from skimage.util import img_as_float
from scipy.optimize import minimize

import pdb

def S(x1,x2,geodesic,sigma_clr=10):
	return math.exp(-pow(geodesic[x1,x2],2)/(2*sigma_clr*sigma_clr))

def compute_saliency_cost(smoothness,w_bg,wCtr):
	n = len(w_bg)
	A = np.zeros((n,n))
	b = np.zeros((n))

	for x in range(0,n):
		A[x,x] = 2 * w_bg[x] + 2 * (wCtr[x])
		b[x] = 2 * wCtr[x]
		for y in range(0,n):
			A[x,x] += 2 * smoothness[x,y]
			A[x,y] -= 2 * smoothness[x,y]
	
	x = np.linalg.solve(A, b)

	return x

def path_length(path,G):
	dist = 0.0
	for i in range(1,len(path)):
		dist += G[path[i - 1]][path[i]]['weight']
	return dist

def make_graph(grid):
	# get unique labels
	vertices = np.unique(grid) - 1 # segment 된 라벨들 (1, 2, 3, 4, ...)

	# map unique labels to [1,...,num_labels]
	# reverse_dict = dict(zip(vertices,np.arange(len(vertices)))) # 1을 0으로, 2를 1로, 3을 2로 .... 
 	# grid = np.array([reverse_dict[x] for x in grid.flat]).reshape(grid.shape) 
	grid = grid - 1

	# create edges
	down = np.c_[grid[:-1, :].ravel(), grid[1:, :].ravel()] # 두개의 1차원 배열을 column으로 세로로 붙어 2차원 배열 만들기
	right = np.c_[grid[:, :-1].ravel(), grid[:, 1:].ravel()] # 위아래 근접 좌우 근접 조합을 만드는 것임
	all_edges = np.vstack([right, down]) # 그 조합들을 합침
	all_edges = all_edges[all_edges[:, 0] != all_edges[:, 1], :] # 조합된 두개의 근접 조합의 vertice가 같지 않은 것들만 남김
	all_edges = np.sort(all_edges,axis=1) # 조합들을 오름차순으로 정리
	num_vertices = len(vertices)
	edge_hash = all_edges[:,0] + num_vertices * all_edges[:, 1] # 작은 값 + vertice 수 * 큰값으로 고유한 어떤 값으로 조합을 정의함
	# find unique connections
	edges = np.unique(edge_hash)
	# undo hashing
	edges = [[vertices[x%num_vertices],
			  vertices[x//num_vertices]] for x in edges] 
 
	return vertices, edges
	

def get_saliency_rbd(img_path):

	# Saliency map calculation based on:
	# Saliency Optimization from Robust Background Detection, Wangjiang Zhu, Shuang Liang, Yichen Wei and Jian Sun, IEEE Conference on Computer Vision and Pattern Recognition (CVPR), 2014

	img = skimage.io.imread(img_path)

	if len(img.shape) != 3: # got a grayscale image
		img = skimage.color.gray2rgb(img)

	img_lab = img_as_float(skimage.color.rgb2lab(img))

	img_rgb = img_as_float(img)

	img_gray = img_as_float(skimage.color.rgb2gray(img))
    
	# slic = Slic(num_components=1600, compactness=10)
	# segments_slic = slic.iterate(img) # Cluster Map
	segments_slic = slic(img_rgb, n_segments=250, compactness=10, sigma=1, enforce_connectivity=False)

	num_segments = len(np.unique(segments_slic))

	nrows, ncols = segments_slic.shape
	max_dist = math.sqrt(nrows*nrows + ncols*ncols)

	grid = segments_slic

	(vertices,edges) = make_graph(grid)
	grid = grid - 1
	gridx, gridy = np.mgrid[:grid.shape[0], :grid.shape[1]]

	centers = dict()
	colors = dict()
	distances = dict()
	boundary = dict()

	for v in vertices:
		centers[v] = [gridy[grid == v].mean(), gridx[grid == v].mean()]
		colors[v] = np.mean(img_lab[grid==v],axis=0)

		x_pix = gridx[grid == v]
		y_pix = gridy[grid == v]

		if np.any(x_pix == 0) or np.any(y_pix == 0) or np.any(x_pix == nrows - 1) or np.any(y_pix == ncols - 1):
			boundary[v] = 1
		else:
			boundary[v] = 0

	G = nx.Graph()

	#buid the graph
	for edge in edges:
		pt1 = edge[0]
		pt2 = edge[1]
		color_distance = scipy.spatial.distance.euclidean(colors[pt1],colors[pt2])
		G.add_edge(pt1, pt2, weight=color_distance )

	#add a new edge in graph if edges are both on boundary
	for v1 in vertices:
		if boundary[v1] == 1:
			for v2 in vertices:
				if boundary[v2] == 1:
					color_distance = scipy.spatial.distance.euclidean(colors[v1],colors[v2])
					G.add_edge(v1,v2,weight=color_distance)

	geodesic = np.zeros((len(vertices),len(vertices)),dtype=float)
	spatial = np.zeros((len(vertices),len(vertices)),dtype=float)
	smoothness = np.zeros((len(vertices),len(vertices)),dtype=float)
	adjacency = np.zeros((len(vertices),len(vertices)),dtype=float)

	sigma_clr = 10.0
	sigma_bndcon = 1.0
	sigma_spa = 0.25
	mu = 0.1

	all_shortest_paths_color = nx.shortest_path(G,source=None,target=None,weight='weight')

	for v1 in vertices:
		for v2 in vertices:
			if v1 == v2:
				geodesic[v1,v2] = 0
				spatial[v1,v2] = 0
				smoothness[v1,v2] = 0
			else:
				geodesic[v1,v2] = path_length(all_shortest_paths_color[v1][v2],G)
				spatial[v1,v2] = scipy.spatial.distance.euclidean(centers[v1],centers[v2]) / max_dist
				smoothness[v1,v2] = math.exp( - (geodesic[v1,v2] * geodesic[v1,v2])/(2.0*sigma_clr*sigma_clr)) + mu 

	for edge in edges:
		pt1 = edge[0]
		pt2 = edge[1]
		adjacency[pt1,pt2] = 1
		adjacency[pt2,pt1] = 1

	for v1 in vertices:
		for v2 in vertices:
			smoothness[v1,v2] = adjacency[v1,v2] * smoothness[v1,v2]

	area = dict()
	len_bnd = dict()
	bnd_con = dict()
	w_bg = dict()
	ctr = dict()
	wCtr = dict()

	for v1 in vertices:
		area[v1] = 0
		len_bnd[v1] = 0
		ctr[v1] = 0
		for v2 in vertices:
			d_app = geodesic[v1,v2]
			d_spa = spatial[v1,v2]
			w_spa = math.exp(- ((d_spa)*(d_spa))/(2.0*sigma_spa*sigma_spa))
			area_i = S(v1,v2,geodesic)
			area[v1] += area_i
			len_bnd[v1] += area_i * boundary[v2]
			ctr[v1] += d_app * w_spa
		bnd_con[v1] = len_bnd[v1] / math.sqrt(area[v1])
		w_bg[v1] = 1.0 - math.exp(- (bnd_con[v1]*bnd_con[v1])/(2*sigma_bndcon*sigma_bndcon))

	for v1 in vertices:
		wCtr[v1] = 0
		for v2 in vertices:
			d_app = geodesic[v1,v2]
			d_spa = spatial[v1,v2]
			w_spa = math.exp(- (d_spa*d_spa)/(2.0*sigma_spa*sigma_spa))
			wCtr[v1] += d_app * w_spa *  w_bg[v2]

	# normalise value for wCtr

	min_value = min(wCtr.values())
	max_value = max(wCtr.values())

	minVal = [key for key, value in wCtr.items() if value == min_value]
	maxVal = [key for key, value in wCtr.items() if value == max_value]

	for v in vertices:
		wCtr[v] = (wCtr[v] - min_value)/(max_value - min_value)

	img_disp1 = img_gray.copy()
	img_disp2 = img_gray.copy()

	x = compute_saliency_cost(smoothness,w_bg,wCtr)

	for v in vertices:
		img_disp1[grid == v] = x[v]

	img_disp2 = img_disp1.copy()
	sal = np.zeros((img_disp1.shape[0],img_disp1.shape[1],3))

	sal = img_disp2
	sal_max = np.max(sal)
	sal_min = np.min(sal)
	sal = 255 * ((sal - sal_min) / (sal_max - sal_min))

	return sal

def get_saliency_ft(img_path):

	# Saliency map calculation based on:

	img = skimage.io.imread(img_path)

	img_rgb = img_as_float(img)

	img_lab = skimage.color.rgb2lab(img_rgb) 

	mean_val = np.mean(img_rgb,axis=(0,1))

	kernel_h = (1.0/16.0) * np.array([[1,4,6,4,1]])
	kernel_w = kernel_h.transpose()

	blurred_l = scipy.signal.convolve2d(img_lab[:,:,0],kernel_h,mode='same')
	blurred_a = scipy.signal.convolve2d(img_lab[:,:,1],kernel_h,mode='same')
	blurred_b = scipy.signal.convolve2d(img_lab[:,:,2],kernel_h,mode='same')

	blurred_l = scipy.signal.convolve2d(blurred_l,kernel_w,mode='same')
	blurred_a = scipy.signal.convolve2d(blurred_a,kernel_w,mode='same')
	blurred_b = scipy.signal.convolve2d(blurred_b,kernel_w,mode='same')

	im_blurred = np.dstack([blurred_l,blurred_a,blurred_b])

	sal = np.linalg.norm(mean_val - im_blurred,axis = 2)
	sal_max = np.max(sal)
	sal_min = np.min(sal)
	sal = 255 * ((sal - sal_min) / (sal_max - sal_min))
	return sal


img = "/content/data/semi_sup_data/CWFID/num30/test/input/001_image.png"
rbd = get_saliency_rbd(img).astype('uint8')
fig = plt.figure() 
ax = fig.add_subplot(1, 1, 1) 
ax.imshow(rbd, cmap='gray') 
plt.axis("off") 
plt.show()

In [ ]:
np.set_printoptions(threshold=np.inf, linewidth=np.inf)
print(rbd)


In [5]:
from skimage.segmentation import slic
from fast_slic import Slic
import skimage
img_path = "/content/data/semi_sup_data/CWFID/num30/test/input/001_image.png"
img = skimage.io.imread(img_path)

if len(img.shape) != 3: # got a grayscale image
    img = skimage.color.gray2rgb(img)
img_rgb = img_as_float(img)
slic_f = Slic(num_components=250, compactness=10, min_size_factor=0)



In [6]:
fast_slic = slic_f.iterate(img) # Cluster Map

In [10]:
sk_slic = slic(img_rgb, n_segments=250, compactness=10, sigma=1, enforce_connectivity=False, start_label=0)

In [11]:
fast_slic

array([[  0,   0,   0, ...,  16,  16,  16],
       [  0,   0,   0, ...,  16,  16,  16],
       [  0,   0,   0, ...,  16,  16,  16],
       ...,
       [247, 247, 247, ..., 238, 238, 238],
       [247, 247, 247, ..., 238, 238, 238],
       [247, 247, 247, ..., 238, 238, 238]], dtype=int16)

In [12]:
sk_slic

array([[  0,   0,   0, ...,  17,  17,  17],
       [  0,   0,   0, ...,  17,  17,  17],
       [  0,   0,   0, ...,  17,  17,  17],
       ...,
       [234, 234, 234, ..., 251, 251, 251],
       [234, 234, 234, ..., 233, 233, 233],
       [234, 234, 234, ..., 233, 233, 233]])